In [1]:
# Distruct 3: 1.275896	103.812776
# District 7: 1.295695	103.856892
# District 20: 1.366447	103.854150

In [18]:
import requests
import json
import pandas as pd
from getpass import getpass
import dotenv 
import folium
from folium import Choropleth, Circle, Marker, Icon, Map, TileLayer
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd

In [3]:
token = getpass()

,Category,Count
0,Schools,10
1,Starbucks,0
2,Night Clubs,10
3,Vegan Restaurants,10
4,Basketball Courts,10
5,Pet Grooming Services,10


,Category,Count
0,Schools,10
1,Starbucks,0
2,Night Clubs,2
3,Vegan Restaurants,6
4,Basketball Courts,3
5,Pet Grooming Services,3


In [14]:
list_dict_locations = [
        {"name": "District 20", "lat": "1.366447", "long": "103.854150"},
        {"name": "District 7", "lat": "1.295695", "long": "103.856892"},
        {"name": "District 3", "lat": "1.275896", "long": "103.812776"}
]


In [15]:
def get_foursquare_data():
    base_url = "https://api.foursquare.com/v3/places/search"
    locations = (list_dict_locations)
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    data_frames = []
    for location in locations:
        links = [
            f"query=school&ll={location['lat']}%2C{location['long']}&radius=1000",
            f"ll={location['lat']}%2C{location['long']}&radius=500&chains=ab4c54c0-d68a-012e-5619-003048cad9da",
            f"ll={location['lat']}%2C{location['long']}&radius=1000&categories=10032",
            f"ll={location['lat']}%2C{location['long']}&radius=500&categories=13377",
            f"ll={location['lat']}%2C{location['long']}&radius=10000&categories=18008",
            f"ll={location['lat']}%2C{location['long']}&radius=1000&categories=11134"
        ]
        counts = []
        for link in links:
            url = f"{base_url}?{link}"
            response = requests.get(url, headers=headers)
            count = len(response.json().get("results", []))
            counts.append(count)
        df = pd.DataFrame({"Category": ["Schools 1km", "Starbucks 500m", "Night Clubs 1km", "Vegan Restaurants 500m", "Basketball Courts 10km", "Pet Grooming Services 1km"], "Count": counts})
        df["Location"] = location["name"]
        data_frames.append(df)
    return pd.concat(data_frames).pivot(index="Location", columns="Category", values="Count")


In [16]:
dataframe_best_district = get_foursquare_data()
dataframe_best_district

In [17]:
dataframe_best_district

Category,Basketball Courts 10km,Night Clubs 1km,Pet Grooming Services 1km,Schools 1km,Starbucks 500m,Vegan Restaurants 500m
Location,,,,,,
District 20,10,2,3,10,0,6
District 3,10,0,0,10,0,0
District 7,10,10,1,10,10,7


In [19]:
signapore_map = Map(location = [1.287953, 103.851784], zoom_start = 11)

for index, row in df.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']], tooltip=row['Name']).add_to(signapore_map)

folium.Choropleth(
    geo_data=geo_sgp,
    data=df,
    columns=df.columns,
    key_on="feature.properties.name",
).add_to(signapore_map)


NameError: name 'df' is not defined